# 🧪 Pruebas del Servicio: IA Validador Multimodal Inteligente de Archivos

## 📌 Propósito del Notebook

Este notebook tiene como objetivo probar de manera automatizada el funcionamiento del servicio **IA Validador Multimodal Inteligente de Archivos**, una solución desarrollada por el equipo de Ciencia de Datos e Inteligencia Artificial de la OTI de la Superintendencia de Industria y Comercio (SIC).

El servicio permite validar archivos cargados por los ciudadanos en el formulario web de la SIC, evaluando su calidad y estructura. El sistema detecta automáticamente el tipo de archivo (`pdf`, `docx`, `xlsx`, `txt`), lo enruta al microservicio correspondiente y devuelve una respuesta estructurada que incluye:

- ✅ Formato detectado
- 🧠 Validaciones realizadas (estructura, peso, contenido, errores)
- ⏱️ Tiempo de inicio y fin de la validación
- ⚠️ Tipos de errores detectados (corrupción, vacío, liviano, peso excedido, errores inesperados)
- 📄 Información específica del archivo (páginas, párrafos, líneas, caracteres, etc.)

## 🗂️ Estructura del Notebook

1. **Configuración inicial**
   - Cargar librerías
   - Definir constantes de conexión al servicio gateway
   - Definir funciones auxiliares

2. **Carga de archivos de ejemplo**
   - Leer archivos desde una carpeta local
   - Mostrar nombres de archivo a procesar

3. **Envío de archivos al gateway**
   - Detectar tipo de archivo automáticamente
   - Enviar solicitud a `/validate-file` vía HTTP
   - Almacenar la respuesta estructurada

4. **Visualización de resultados**
   - Conversión de respuestas a `pandas.DataFrame`
   - Análisis de errores por tipo
   - Visualización tabular de respuestas por archivo

## 📦 Microservicios involucrados

- `gateway_service`: Encargado de detectar el tipo de archivo y redirigirlo.
- `pdf_service`: Valida archivos PDF (estructura, páginas, peso).
- `docx_service`: Valida documentos Word.
- `xlsx_service`: Valida archivos Excel.
- `txt_service`: Valida archivos de texto plano.

---

Este notebook es parte del proceso de validación funcional y de calidad del sistema en su versión **beta**, y permite anticipar mejoras, detectar errores de integración, y verificar que el procesamiento y retorno de información se realiza correctamente.


## 🧰 Importación de módulos necesarios

Se importan los siguientes módulos para llevar a cabo las pruebas del servicio `IA Validador Multimodal Inteligente de Archivos`:

- `requests`: permite realizar peticiones HTTP al microservicio gateway para validar los archivos.
- `pathlib.Path`: facilita el manejo de rutas de archivos de manera multiplataforma.
- `pandas`: utilizado para almacenar y visualizar los resultados de las validaciones en forma de tabla.
- `typing.List`: se usa para anotar listas en las definiciones de funciones, mejorando la legibilidad y mantenibilidad del código.


In [10]:
# IA_Validador_Test.ipynb (código para una celda de Jupyter Notebook)

import requests
from pathlib import Path
import pandas as pd
from typing import List


In [11]:
import plotly.express as px
import plotly.graph_objects as go


## ⚙️ Funciones auxiliares para las pruebas
En esta sección se definen funciones reutilizables que permiten automatizar el flujo de pruebas del servicio IA Validador Multimodal Inteligente de Archivos a través del microservicio gateway.

In [12]:

def listar_nombres_archivos(directorio: str) -> List[str]:
    """
    Retorna una lista con los nombres de todos los archivos (no carpetas) dentro de un directorio dado.

    Args:
        directorio (str): Ruta al directorio.

    Returns:
        List[str]: Lista con los nombres de los archivos (no rutas completas).
    """
    ruta = Path(directorio)
    if not ruta.exists() or not ruta.is_dir():
        raise ValueError(f"El directorio '{directorio}' no existe o no es válido.")

    return [f.name for f in ruta.iterdir() if f.is_file()]


In [13]:

def validar_archivos_gateway(lista_archivos: List[Path], gateway_url: str) -> List[dict]:
    resultados = []
    
    for archivo in lista_archivos:
        try:
            with open(archivo, "rb") as f:
                files = {"file": (archivo.name, f, "application/octet-stream")}
                response = requests.post(gateway_url, files=files, timeout=60)
                result_json = response.json()
                
                resultados.append({
                    "Tipo de archivo": archivo.suffix.lstrip(".").lower(),
                    "Nombre archivo": archivo.name,
                    "Código HTTP": response.status_code,
                    "FLAG_OK_file": result_json.get("FLAG_OK_file"),
                    "Tamaño (MB)": result_json.get("weight_file"),
                    "Formato detectado": result_json.get("format"),
                    "Mensaje": result_json.get("message", ""),
                    "Respuesta completa": result_json
                })
        except Exception as e:
            resultados.append({
                "Tipo de archivo": archivo.suffix.lstrip(".").lower(),
                "Nombre archivo": archivo.name,
                "Código HTTP": "Error",
                "FLAG_OK_file": False,
                "Tamaño (MB)": None,
                "Formato detectado": None,
                "Mensaje": str(e),
                "Respuesta completa": {}
            })
    
    return resultados


In [14]:

def resultados_a_dataframe(resultados: list) -> pd.DataFrame:
    filas = []
    for r in resultados:
        fila = {
            "Tipo de archivo": r.get("Tipo de archivo"),
            "Nombre archivo": r.get("Nombre archivo"),
            "Código HTTP": r.get("Código HTTP"),
            "FLAG_OK_file": r.get("FLAG_OK_file"),
            "Tamaño (MB)": r.get("Tamaño (MB)"),
            "Formato detectado": r.get("Formato detectado"),
            "Mensaje": r.get("Mensaje"),
        }

        respuesta = r.get("Respuesta completa", {})
        fila.update({
            "Páginas/Parrafos/Lineas": respuesta.get("pages") or respuesta.get("paragraph_count") or respuesta.get("line_count"),
            "error_type_01_file": respuesta.get("error_type_01_file"),
            "error_type_02_file": respuesta.get("error_type_02_file"),
            "error_type_03_file": respuesta.get("error_type_03_file"),
            "error_type_04_file": respuesta.get("error_type_04_file"),
            "error_type_09_file": respuesta.get("error_type_09_file"),
            "Inicio validación": respuesta.get("time_val_file_init"),
            "Fin validación": respuesta.get("time_val_file_end"),
        })
        filas.append(fila)

    return pd.DataFrame(filas)


## 🛠️ Configuración de la prueba
En esta sección se definen los parámetros necesarios para ejecutar las pruebas del servicio de validación de archivos

Detalles:
* carpeta: especifica el directorio local donde se almacenan los archivos a validar.
* archivos: genera una lista de archivos encontrados en dicha carpeta.
* GATEWAY_URL: dirección del servicio FastAPI que expone el endpoint /validate-file del microservicio gateway, al cual se envían los archivos.



In [32]:

# Ruta a la carpeta que contiene los archivos de prueba
carpeta = Path("C:\\Users\\anmmu\\OneDrive\\Proyectos\\data\\")

# Obtener todos los archivos
archivos = list(carpeta.glob("*"))

# URL del servicio Gateway
GATEWAY_URL = "http://localhost:8000/validate-file"


## ▶️ Ejecución de las pruebas
En esta etapa se ejecuta la validación de cada archivo contra el servicio Gateway, y posteriormente se organiza la salida en un DataFrame para su análisis.

Resultado esperado:
Una tabla con las siguientes columnas clave:
* Tipo de archivo
* Nombre archivo
* Código HTTP
* FLAG_OK_file
* Tamaño (MB)
* Formato detectado
* Mensaje
* Páginas/Párrafos/Líneas
* Errores específicos
* Tiempos de validación

In [33]:

# Ejecutar la validación
resultados = validar_archivos_gateway(archivos, GATEWAY_URL)

In [34]:
df_resultados = resultados_a_dataframe(resultados)
df_resultados.head()


,Tipo de archivo,Nombre archivo,Código HTTP,FLAG_OK_file,Tamaño (MB),Formato detectado,Mensaje,Páginas/Parrafos/Lineas,error_type_01_file,error_type_02_file,error_type_03_file,error_type_04_file,error_type_09_file,Inicio validación,Fin validación
0,txt,19mb.txt,413,False,18.88,txt,Archivo demasiado grande. Máximo permitido: 10MB,None,False,False,False,True,False,2025-07-06T21:07:51.018677,2025-07-06T21:07:51.063421
1,txt,ascii-art - Copy.txt,502,False,0.00,txt,Error HTTP al contactar microservicio especial...,None,False,False,False,False,True,2025-07-06T21:07:51.090466,2025-07-06T21:07:51.159327
2,txt,ascii-art.txt,502,False,0.00,txt,Error HTTP al contactar microservicio especial...,None,False,False,False,False,True,2025-07-06T21:07:51.165664,2025-07-06T21:07:51.198245
3,docx,Basic management resume.docx,200,True,0.02,docx,Validación exitosa.,None,False,False,False,False,False,2025-07-06T21:07:51.216419,2025-07-06T21:07:51.252291
4,docx,Boletín de regreso a la escuela.docx,200,True,0.94,docx,Validación exitosa.,None,False,False,False,False,False,2025-07-06T21:07:51.260104,2025-07-06T21:07:51.415635


## 📊 Análisis Descriptivo de Resultados
Esta sección permite explorar de forma visual los resultados de la validación de archivos procesados por el servicio IA Validador Multimodal Inteligente de Archivos. Se presentan estadísticas y gráficos interactivos construidos con Plotly para comprender mejor el comportamiento de los diferentes tipos de archivos evaluados.

### ✅ Cantidad de archivos por tipo y validación
Un histograma que muestra cuántos archivos de cada tipo fueron validados correctamente (FLAG_OK_file = True) y cuántos fallaron (False).

### 📦 Tamaño promedio de los archivos por tipo
Un gráfico de barras con el tamaño promedio en megabytes (MB) para cada tipo de archivo. Esto permite detectar archivos sospechosamente livianos que podrían ser inválidos.

### ❗ Distribución de errores detectados
Un gráfico circular que resume los errores más frecuentes identificados por los microservicios:

* error_type_01_file: archivo corrupto
* error_type_02_file: archivo vacío
* error_type_03_file: peso demasiado bajo
* error_type_04_file: peso excedido
* error_type_09_file: error inesperado en microservicio (500, 502, 504, etc.)

### ⏱️ Tiempo de validación por archivo
Un diagrama de caja que muestra la duración (en segundos) que tomó validar cada archivo, segmentado por tipo. Permite detectar diferencias de rendimiento entre microservicios.

In [35]:

# Cantidad de archivos por tipo
fig1 = px.histogram(df_resultados, x="Tipo de archivo", color="FLAG_OK_file", barmode="group",
                    title="Cantidad de archivos por tipo y validación", text_auto=True)
fig1.update_layout(xaxis_title="Tipo de archivo", yaxis_title="Cantidad", legend_title="Validación OK")
fig1.show()

# Tamaño promedio por tipo de archivo
df_tamano = df_resultados.groupby("Tipo de archivo")["Tamaño (MB)"].mean().reset_index()
fig2 = px.bar(df_tamano, x="Tipo de archivo", y="Tamaño (MB)", title="Tamaño promedio de archivo por tipo")
fig2.update_traces(texttemplate='%{y:.2f} MB', textposition='outside')
fig2.update_layout(yaxis_title="Tamaño promedio (MB)")
fig2.show()

# Conteo de errores por tipo
errores = ["error_type_01_file", "error_type_02_file", "error_type_03_file", "error_type_04_file", "error_type_09_file"]
error_counts = df_resultados[errores].sum().reset_index()
error_counts.columns = ["Tipo de error", "Cantidad"]

fig3 = px.pie(error_counts, names="Tipo de error", values="Cantidad", title="Distribución de errores detectados")
fig3.show()

# Tiempos de validación (opcional si necesitas ver diferencias)
df_resultados["Inicio validación"] = pd.to_datetime(df_resultados["Inicio validación"])
df_resultados["Fin validación"] = pd.to_datetime(df_resultados["Fin validación"])
df_resultados["Duración (s)"] = (df_resultados["Fin validación"] - df_resultados["Inicio validación"]).dt.total_seconds()

fig4 = px.box(df_resultados, x="Tipo de archivo", y="Duración (s)", points="all",
              title="Tiempo de validación por tipo de archivo")
fig4.show()
